#Book5_4

# 실습 20. RNN 언어 모델

In [ ]:
# 텍스트 데이터(옛날 이야기) 불러오기

text_data = """옛날 옛적 어느 곳에,
할아버지와 할머니가 살았습니다.
할아버지는 산으로 풀을 베러 갔고,
할머니는 강으로 빨래를 하러 갔습니다.
"""

char_to_int = {char: i for i, char in enumerate(sorted(set(text_data)))}
int_to_char = {i: char for char, i in char_to_int.items()}

encoded = [char_to_int[c] for c in text_data]
decoded = [int_to_char[c] for c in encoded]

print(encoded)
print(decoded)

print(''.join(decoded))

[29, 9, 1, 29, 33, 1, 27, 10, 1, 8, 28, 2, 0, 37, 25, 19, 34, 30, 1, 37, 18, 12, 4, 1, 23, 26, 24, 12, 13, 3, 0, 37, 25, 19, 34, 11, 1, 22, 31, 16, 1, 35, 32, 1, 20, 15, 1, 5, 7, 2, 0, 37, 18, 12, 11, 1, 6, 31, 16, 1, 21, 14, 17, 1, 36, 15, 1, 5, 24, 12, 13, 3, 0]
['옛', '날', ' ', '옛', '적', ' ', '어', '느', ' ', '곳', '에', ',', '\n', '할', '아', '버', '지', '와', ' ', '할', '머', '니', '가', ' ', '살', '았', '습', '니', '다', '.', '\n', '할', '아', '버', '지', '는', ' ', '산', '으', '로', ' ', '풀', '을', ' ', '베', '러', ' ', '갔', '고', ',', '\n', '할', '머', '니', '는', ' ', '강', '으', '로', ' ', '빨', '래', '를', ' ', '하', '러', ' ', '갔', '습', '니', '다', '.', '\n']
옛날 옛적 어느 곳에,
할아버지와 할머니가 살았습니다.
할아버지는 산으로 풀을 베러 갔고,
할머니는 강으로 빨래를 하러 갔습니다.



In [ ]:
# 역자 추가 코딩
print(len(encoded))
print(len(decoded))

60
60


In [ ]:
# 역자 추가 코딩
seq_length = len(text_data.split("\n")[0])
seq_length

13

In [ ]:
# 입력 시퀀스와 타깃 시퀀스 생성

encoded_text = encoded # 일본어판 코딩에 encoded_text가 누락되어 역자가 추가
seq_length = len(text_data.split("\n")[0])

sequences = []
for i in range(len(encoded_text) - seq_length):
  sequences.append(encoded_text[i:i + seq_length + 1])

print(f"시퀀스 길이:{seq_length}")
print(f"레코드 수:{len(sequences)}")
print(f"샘플(처음 3건):\n{sequences[:3]}")

시퀀스 길이:12
레코드 수:61
샘플(처음 3건):
[[29, 9, 1, 29, 33, 1, 27, 10, 1, 8, 28, 2, 0], [9, 1, 29, 33, 1, 27, 10, 1, 8, 28, 2, 0, 37], [1, 29, 33, 1, 27, 10, 1, 8, 28, 2, 0, 37, 25]]


In [ ]:
import numpy as np

# 입력 데이터와 타깃 데이터 생성
X = np.array([seq[:-1] for seq in sequences])
y = np.array([seq[1:] for seq in sequences])

print(X.shape)
print(y.shape)
print(X[:5])
print(y[:5])

(61, 12)
(61, 12)
[[29  9  1 29 33  1 27 10  1  8 28  2]
 [ 9  1 29 33  1 27 10  1  8 28  2  0]
 [ 1 29 33  1 27 10  1  8 28  2  0 37]
 [29 33  1 27 10  1  8 28  2  0 37 25]
 [33  1 27 10  1  8 28  2  0 37 25 19]]
[[ 9  1 29 33  1 27 10  1  8 28  2  0]
 [ 1 29 33  1 27 10  1  8 28  2  0 37]
 [29 33  1 27 10  1  8 28  2  0 37 25]
 [33  1 27 10  1  8 28  2  0 37 25 19]
 [ 1 27 10  1  8 28  2  0 37 25 19 34]]


In [ ]:
import tensorflow as tf

# 모델 정의
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(len(char_to_int), 64, input_length=seq_length),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.Dense(len(char_to_int), activation="softmax")
])

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 12, 64)            2432      
                                                                 
 lstm (LSTM)                 (None, 12, 128)           98816     
                                                                 
 dense (Dense)               (None, 12, 38)            4902      
                                                                 
Total params: 106150 (414.65 KB)
Trainable params: 106150 (414.65 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
# 모델 컴파일
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")

# 모델 훈련
model.fit(X, y, epochs=100, batch_size=64)

Epoch 1/100
1/1 [==============================] - 2s 2s/step - loss: 3.5795
Epoch 2/100
1/1 [==============================] - 0s 13ms/step - loss: 3.5688
Epoch 3/100
1/1 [==============================] - 0s 11ms/step - loss: 3.5560
Epoch 4/100
1/1 [==============================] - 0s 11ms/step - loss: 3.5408
Epoch 5/100
1/1 [==============================] - 0s 12ms/step - loss: 3.5225
Epoch 6/100
1/1 [==============================] - 0s 10ms/step - loss: 3.5007
Epoch 7/100
1/1 [==============================] - 0s 11ms/step - loss: 3.4755
Epoch 8/100
1/1 [==============================] - 0s 11ms/step - loss: 3.4492
Epoch 9/100
1/1 [==============================] - 0s 10ms/step - loss: 3.4304
Epoch 10/100
1/1 [==============================] - 0s 10ms/step - loss: 3.4327
Epoch 11/100
1/1 [==============================] - 0s 11ms/step - loss: 3.4320
Epoch 12/100
1/1 [==============================] - 0s 10ms/step - loss: 3.4174
Epoch 13/100
1/1 [==============================] -

In [ ]:
# 생성하고자 하는 텍스트의 초기 시퀀스 지정
initial_text = "옛날 옛적 어느 곳에,"

# 텍스트 생성
max_output_tokens= 16
for _ in range(max_output_tokens):
  encoded_initial_text = [char_to_int[char] for char in initial_text]
  predicted_char = model.predict(
      np.array([encoded_initial_text]),verbose=0)[:, -1]

  next_char = int_to_char[np.argmax(predicted_char)]
  initial_text += next_char

print(initial_text)


옛날 옛적 어느 곳에,
할아버지와 할머니니가 살았습


In [ ]:
# 특정 국가(미국) 입력문을 5회씩 반복해서 입력한 코드로 책에 실린 코드임

import tensorflow as tf
import numpy as np

# 텍스트 데이터 불러오기
# 참고 각 문자 뒤에 스페이스 들어가면 에러 발생
text_data = """Q.영국의 수도는?
A.런던입니다.
Q.미국의 수도는?
A.Washington DC입니다.
Q.미국의 수도는?
A.Washington DC입니다.
Q.미국의 수도는?
A.Washington DC입니다.
Q.미국의 수도는?
A.Washington DC입니다.
Q.한국의 수도는?
A.서울입니다.
Q.아메리카의 수도는?
A.워싱턴 DC입니다。
Q.미국의 수도는?
A.Washington DC입니다.
Q.잉글랜드의 수도는?
A.런던입니다.
"""
char_to_int = {char: i for i, char in enumerate(sorted(set(text_data)))}
int_to_char = {i: char for char, i in char_to_int.items()}

# 텍스트 데이터를 정수열로 변환
encoded_text = [char_to_int[char] for char in text_data]

# 입력 시퀀스와 타깃 시퀀스 만들기
seq_length = len(text_data.split("\n")[0])

sequences = []

for i in range(len(encoded_text) - seq_length):
  sequences.append(encoded_text[i:i + seq_length + 1])

# 입력 데이터와 목표 데이터 생성
X = np.array([seq[:-1] for seq in sequences])
y = np.array([seq[1:] for seq in sequences])

# 모델 정의
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(len(char_to_int), 64, input_length=seq_length),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.Dense(len(char_to_int), activation="softmax")
])

# 모델 컴파일
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")

# 모델 훈련
model.fit(X, y, epochs=50, batch_size=1)

# 생성할 텍스트의 초기 시퀀스를 지정
initial_text = "Q.미국의 수도는?"

# 텍스트 생성
max_output_tokens= 16
for _ in range(max_output_tokens):
  encoded_initial_text = [char_to_int[char] for char in initial_text]
  predicted_char = model.predict(np.array([encoded_initial_text]),verbose=0)[:, -1]

  next_char = int_to_char[np.argmax(predicted_char)]
  initial_text += next_char

print(initial_text)


Epoch 1/50
233/233 [==============================] - 3s 5ms/step - loss: 2.6844
Epoch 2/50
233/233 [==============================] - 1s 4ms/step - loss: 1.1714
Epoch 3/50
233/233 [==============================] - 1s 4ms/step - loss: 0.5878
Epoch 4/50
233/233 [==============================] - 1s 4ms/step - loss: 0.3615
Epoch 5/50
233/233 [==============================] - 1s 5ms/step - loss: 0.2689
Epoch 6/50
233/233 [==============================] - 1s 4ms/step - loss: 0.2195
Epoch 7/50
233/233 [==============================] - 1s 4ms/step - loss: 0.1871
Epoch 8/50
233/233 [==============================] - 1s 4ms/step - loss: 0.1696
Epoch 9/50
233/233 [==============================] - 1s 4ms/step - loss: 0.1576
Epoch 10/50
233/233 [==============================] - 1s 4ms/step - loss: 0.1508
Epoch 11/50
233/233 [==============================] - 1s 4ms/step - loss: 0.1481
Epoch 12/50
233/233 [==============================] - 1s 4ms/step - loss: 0.1418
Epoch 13/50
233/233 [====

# 실습 21. 트랜스포머를 이용한 신경망 기계번역

In [ ]:
import transformers
transformers.__version__

'4.38.2'

In [ ]:
from transformers import MarianMTModel, MarianTokenizer

In [ ]:
# 한국어판 코드
# 사전학습 모델 및 토크나이저 불러오기
model_name = "Helsinki-NLP/opus-mt-ko-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
model

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(65001, 512, padding_idx=65000)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(65001, 512, padding_idx=65000)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLU()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05

In [ ]:
def translate_korean_to_english(text):

  # 입력 텍스트 토크나이징
  inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

  # 모델로 번역 실행
  outputs = model.generate(**inputs)

  # 모델 결과를 디코딩
  translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
  return translated_text

In [ ]:
# 번역 예
korean_text = "오늘은 날씨가 좋네요."
english_translation = translate_korean_to_english(korean_text)

print("한국어:", korean_text)
print("영어:", english_translation)

한국어: 오늘은 날씨가 좋네요.
영어: It's a good day.


In [ ]:
# 번역 예
korean_text = "Python는 영어와 같은 프로그래밍 언어이기 때문에 말하듯이 쓸 수 있어 매우 편리하다. 나는 파이썬을 처음 사용한 지 벌써 15년 가까이 되었다."
english_translation = translate_korean_to_english(korean_text)

print("한국어:", korean_text)
print("영어:", english_translation)

한국어: Python는 영어와 같은 프로그래밍 언어이기 때문에 말하듯이 쓸 수 있어 매우 편리하다. 나는 파이썬을 처음 사용한 지 벌써 15년 가까이 되었다.
영어: Python is a programming language like English, so it's very convenient to write it like this. I've been using Python for the first time for almost 15 years.
